1. We have a peptidome $p_i \in P$
2. We can sample from this peptidome $\hat{p_i} \sim P$
3. We want to create a graph $G$ that generates* $\hat{p_i}$. To do this:
    - draw $\hat{p_i} \sim P$
    - Generate peptidome from $G$. Calculate KL-divergence $\delta$ between generated peptidome and $\hat{p_i}$
    - for pair $(u_i, u_j)$ where $u_i \in u_j$ in $\hat{p_i}$. Add $\delta \times u_i$ to $w_{i,j}$ where $w_{i,j}$ is the weight of the edge between the nodes. (Kind of)
    - Penalize reliance on a single node by $\epsilon \times \sum_i{w_{i,j}}$ where $\epsilon$ is some penalty factor.
    


*To generate a peptidome from $G$. Let $N_T$ be the number of peptides in $\hat{p_i}$. Start from the largest peptide. The peptide is cleaved with probability $w_{i,j}$. Iterate until number of peptides $\geq N_T$.

In [6]:
import numpy as np

# Generate synthetic sample
P = {"ABCD": 100, "ABC": 20, "BCD": 5, "BC":5, "CD":1, "C":0, "D":0, "AB": 50, "A": 5}
keys = list(P.keys())
values = list(P.values())
probs = [v / sum(values) for v in values]

n_sample = 50
p_hat = np.random.choice(keys, n_sample, replace=True, p=probs)

p_hat_dict = {k:0 for k in keys}
for key in p_hat:
    p_hat_dict[key] += 1

p_hat_dict

{'ABCD': 28,
 'ABC': 2,
 'BCD': 2,
 'BC': 3,
 'CD': 0,
 'C': 0,
 'D': 0,
 'AB': 13,
 'A': 2}

In [2]:
import networkx as nx

G = nx.DiGraph()
G.add_nodes_from(keys)
for key1 in keys:
    for key2 in keys:
        if key1 in key2 and key1 != key2: #key 1 = ABC, key 2 = ABCD
            G.add_edge(key2, key1, weight=1)
G.edges(data=True)

OutEdgeDataView([('ABCD', 'ABC', {'weight': 1}), ('ABCD', 'BCD', {'weight': 1}), ('ABCD', 'BC', {'weight': 1}), ('ABCD', 'CD', {'weight': 1}), ('ABCD', 'C', {'weight': 1}), ('ABCD', 'D', {'weight': 1}), ('ABCD', 'AB', {'weight': 1}), ('ABCD', 'A', {'weight': 1}), ('ABC', 'BC', {'weight': 1}), ('ABC', 'C', {'weight': 1}), ('ABC', 'AB', {'weight': 1}), ('ABC', 'A', {'weight': 1}), ('BCD', 'BC', {'weight': 1}), ('BCD', 'CD', {'weight': 1}), ('BCD', 'C', {'weight': 1}), ('BCD', 'D', {'weight': 1}), ('BC', 'C', {'weight': 1}), ('CD', 'C', {'weight': 1}), ('CD', 'D', {'weight': 1}), ('AB', 'A', {'weight': 1})])

In [3]:
N_T = sum(values)
longest_key = sorted(keys, key=len)[-1]

p_generated = {key:0 for key in keys}


N_T_hat = 0
while N_T_hat < N_T:
    p_generated[longest_key] += 1
    for sequence, copy_number in p_generated.items():
        if copy_number == 0:
            continue
        out_edges = G.out_edges(sequence, data=True)
        weights = np.array([weight["weight"] for _,_, weight in out_edges])
        sum_w = sum(weights)
        weights = weights/sum_w
        edges_to = [edge_to for _, edge_to, _ in out_edges]
        for w, e in zip(weights, edges_to):
            p_generated[e] += w*copy_number
    N_T_hat = sum(p_generated.values())
p_generated

{'ABCD': 9,
 'ABC': 5.625,
 'BCD': 5.625,
 'BC': 15.9375,
 'CD': 10.78125,
 'C': 85.546875,
 'D': 28.828125,
 'AB': 10.78125,
 'A': 46.875}

In [4]:
def KL(a, b):
    a = np.asarray(list(a)) 
    b = np.asarray(list(b)) 
    a = a / np.sum(a)
    b = b / np.sum(b)

    return np.sum(np.where(a != 0, a * np.log(a / b), 0))

kl = KL(p_hat_dict.values(), p_generated.values())
print(kl)
lr = 0.001 # learning rate
# time to update the graph G

for key in keys:
    out_edges = G.out_edges(key, data=True)
    for _, target, weight in out_edges:
        origin_copy_number = p_hat_dict[key]
        target_copy_number = p_hat_dict[target]
        generated_target_copy_number = p_generated[target]
        diff = target_copy_number - generated_target_copy_number
        new_weight = max(0, weight["weight"] + (diff*lr*kl)) # weight cannot be less than 0
        nx.set_edge_attributes(G, {(key, target):{'weight':new_weight}})
G.edges(data=True)

1.9162457771105468


/var/folders/64/ytcycnwx6w936vfxs5_7q3hm0000gn/T/ipykernel_19693/1200294349.py:7: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.where(a != 0, a * np.log(a / b), 0))
/var/folders/64/ytcycnwx6w936vfxs5_7q3hm0000gn/T/ipykernel_19693/1200294349.py:7: RuntimeWarning: invalid value encountered in multiply
  return np.sum(np.where(a != 0, a * np.log(a / b), 0))


OutEdgeDataView([('ABCD', 'ABC', {'weight': 1.0045510837206375}), ('ABCD', 'BCD', {'weight': 0.9911373632808638}), ('ABCD', 'BC', {'weight': 0.9732923244815217}), ('ABCD', 'CD', {'weight': 0.9793404752155269}), ('ABCD', 'C', {'weight': 0.8360711620362462}), ('ABCD', 'D', {'weight': 0.944758227206735}), ('ABCD', 'AB', {'weight': 1.0023354245408536}), ('ABCD', 'A', {'weight': 0.9120922249750537}), ('ABC', 'BC', {'weight': 0.9732923244815217}), ('ABC', 'C', {'weight': 0.8360711620362462}), ('ABC', 'AB', {'weight': 1.0023354245408536}), ('ABC', 'A', {'weight': 0.9120922249750537}), ('BCD', 'BC', {'weight': 0.9732923244815217}), ('BCD', 'CD', {'weight': 0.9793404752155269}), ('BCD', 'C', {'weight': 0.8360711620362462}), ('BCD', 'D', {'weight': 0.944758227206735}), ('BC', 'C', {'weight': 0.8360711620362462}), ('CD', 'C', {'weight': 0.8360711620362462}), ('CD', 'D', {'weight': 0.944758227206735}), ('AB', 'A', {'weight': 0.9120922249750537})])